In [84]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import sqlite3
from datetime import date
import pandas as pd
from collections import Counter 

## Mobile, Alabama News Sources

In [2]:
def al_data_title_scraper():
    al_data = urlopen('https://www.al.com/')
    al_soup = BeautifulSoup(al_data, 'html.parser')
    articles = al_soup.find_all('article')
    regex_pattern = r'\d+[hm]\sago'
    titles = []
    
    for i in articles:
        titles.append(i.text)
        
    titles = [x.strip(regex_pattern).lstrip().rstrip() for x in titles]
    titles = [i for i in set(titles)]
    
    return titles

In [3]:
def fox_10_news_scraper():
    fox_10_news = urlopen('https://www.fox10tv.com/')
    fox_10_news_soup = BeautifulSoup(fox_10_news, 'html.parser')
    fox_titles = fox_10_news_soup.find_all('a', class_='tnt-asset-link')
    remove_list = ['Tracking the Tropics', '1st & 10','COVID-19 Headlines', 'Back to School', 'News Now Update', 'Our Apps', 'News Now Livestream']
    regex_pattern = r'\d:\d\d'
    
    local_fox_titles = []
    for i in fox_titles:
        local_fox_titles.append(i.text.rstrip().lstrip())
        
    final_fox_titles = []
    for i in local_fox_titles:
        if i == '' or re.match(regex_pattern, i):
            pass
        elif i in remove_list:
            pass
        else:
            final_fox_titles.append(i)
            
    return final_fox_titles

## Birmingham, Alabama News Scrapers

In [4]:
### Birmingham, Alabama
def wbrc_scraper():
    wbrc_birmingham = urlopen('https://www.wbrc.com/')
    wbrc_soup = BeautifulSoup(wbrc_birmingham, 'html.parser')
    new_titles = wbrc_soup.find_all('a',class_='unstyled-link')
    
    cleaned_titles = []
    
    for i in new_titles:
        cleaned_titles.append(i.text.rstrip().lstrip())
        
    return cleaned_titles

In [5]:
def al_birmingham_scraper():
    al_birmingham = urlopen('https://www.al.com/birmingham/')
    al_soup = BeautifulSoup(al_birmingham, 'html.parser')
    articles = al_soup.find_all('article')

    for i in articles:
        regex_pattern = r'\d+[hm]\sago'
        titles = []
    
        for i in articles:
            titles.append(i.text)
        
        titles = [x.strip(regex_pattern).lstrip().rstrip() for x in titles]
        titles = [i for i in set(titles)]
    
        return titles

### Huntsville, Alabama - News Scrapers

In [ ]:
### Grab news from Huntsville


### Database Functions (Will be moved)

In [6]:
def show_all_data():
    con = sqlite3.connect('news.db')
    cursor = con.cursor()
    cursor.execute('SELECT * from news_titles')
    titles = cursor.fetchall()
    con.close()
    return titles

In [41]:
def grab_titles():
    title_list = []
    dirty_data = pd.Series(show_headlines())
    for i in dirty_data:
        title_list.append(i[0])
    return pd.Series(title_list)

In [7]:
def show_headlines():
    con = sqlite3.connect('news.db')
    cursor = con.cursor()
    cursor.execute('SELECT article_title from news_titles')
    titles = cursor.fetchall()
    con.close()
    return titles

In [8]:
### Be sure to update weekday, or find an option that makes the days of the week automatically update

def add_headlines(titles, source):
    today = date.today()
    weekday = 'Monday'
    for i in titles:
        con = sqlite3.connect('news.db')
        cursor = con.cursor()
        cursor.execute('INSERT INTO news_titles (article_title, source, weekday, date) VALUES (?,?,?,?)', (i, source, weekday, today))
        con.commit()
        
    con.close()

In [9]:
def run_all():
    al_headlines_main = al_data_title_scraper()
    add_headlines(al_headlines_main, 'www.al.com')
    fox_10_headlines = fox_10_news_scraper()
    add_headlines(fox_10_headlines, 'www.fox10tv.com')
    wbrc_headlines = wbrc_scraper()
    add_headlines(wbrc_headlines, 'www.wbrc.com/')
    
    return 'Program has sucessfully scraped and saved the data.'

In [10]:
run_all()

'Program has sucessfully scraped and saved the data.'

In [73]:
all_titles = grab_titles()

In [96]:
new_str = ''
for i in all_titles:
    new_str = new_str + i + '\n'

In [97]:
new_str

'Alabama adds 754 confirmed coronavirus casesThe daily update on coronavirus cases in Alabama.5h\nWhitmire: LGBTQ charter school rejection is unacceptable7h\nTrump to announce Supreme Court pick on Friday or SaturdayTrump disparaged reports that Ruth Bader Ginsburg had told her granddaughter it was her wish that a replacement justice not be confirmed until the ...7h\nAlabama reserve cornerback suffers arm injuryHe suffered the injury last week.3h\nThe 2020 Alabama football depth chart3h\nAn update on Joey Gatewood heading into Kentucky-Auburn4h\nCheck out Amari Cooper’s circus catch for Cowboy\nAL.com\'s SEC Insider: Alabama 2020 Season PreviewSEC Insider: Alabama 2020 Season Preview\n- The wildest offseason ever?\n- Offense: Moving on from Tua\n- Defense: Back to the standard?4h\nIncredible last-second scramble play leads to Hillcrest-Tuscaloosa 34-28 win over WetumpkaHillcrest-Tuscaloosa trailed Wetumpka 30-28 with 2.9 seconds left on its own 41-yard line when coach Sam Adams called 

### Observing the Data that we have collected

In [61]:
### There will be a few list that will make up the entire sports spectrum
### For example, there will be football words, baseball words, basketball words, etc.

# football_positions = ['QB','RB','FB','WR','OT','C','G','TE','DE','DT', 'LB', 'OLB', 'S', 'CB', 'FS']

football_words = ['Saban', 'ESPN', 'Alabama football', 'Tide', 'Tide', 'football', 'NFL', 'Auburn football',
                 'Nick Saban', 'cornerback', 'vs.', 'players', 'games', 'College Football']

crime_words = ['murder', 'shooting', 'killed', 'dead', 'injured', 'homicide', 'gunfire', 'injured', 'beaten', 'death',
              'arrested', 'abuse', 'charged']

In [62]:
football_count = 0
crime_count = 0

In [63]:
football_articles = []
crime_articles = []

In [71]:
for i in titles:
    for j in football_words:
        if j in i[0]:
            football_articles.append(i[0])

In [74]:
football_count = len(set(football_articles))
crime_count = len(set(crime_articles))

44

In [75]:
for i in titles:
    for j in crime_words:
        if j in i[0]:
            crime_articles.append(i[0])

- Needs a db, or csv of news_sources, city head-quarter, news area, type_of_news (local, national), etc.
- Search and find the top three cities in each state. Grab news headlines

CREATE TABLE news_sources (id,company_name,news_city,news_state)